Run the notebook at TODAY after 15:00:00 (so the market is closed and data is available).

Input your current position.

The orders are for tomorrow.

In [1]:
import pandas as pd
cal = pd.read_csv('/data/linq/.qlib/qlib_data/cn_data/calendars/day.txt')
TODAY = str(cal.iloc[-1,0])
YESTERDAY = str(cal.iloc[-2,0])
# TODAY = str(cal.iloc[-2,0])
# YESTERDAY = str(cal.iloc[-3,0])

BT_START = '2024-12-01'

print(BT_START)
print(YESTERDAY)
print(TODAY)

2024-12-01
2024-12-10
2024-12-11


In [2]:
# BT_START = '2024-12-01'
# TODAY = '2024-12-09'

test_split = (TODAY, TODAY)
bt_split = (BT_START, YESTERDAY)
pos_txt = f'''\
date {TODAY}
cash 316
SH600570 800
SZ002459 1300
SH600104 1100
SH600588 1500
SH600233 1000
'''



REC_LIST = [
    ('GBDT0', 'EXP_BENCH', 'c745fed5a5ec439f9abeafac76e750a5'), # GBDT BENCH_NOW r0 all
    ('GBDT1', 'EXP_BENCH', 'ffe9470bec09417fb4db91128edc0a87'), # GBDT BENCH_NOW r1 all
    ('GBDT2', 'EXP_BENCH', 'e420ff30870d4e55a1c5e0fb207fa8a0'), # GBDT BENCH_A r0 all
    # ('GBDT3', 'EXP_BENCH', '1dcc7b1ec14e46379690c4f2d6e5242d'), # GBDT BENCH_A r1 all
    ('GBDT4', 'EXP_BENCH', '8d7e72af7f4b475f9f3646021a29bb1e'), # GBDT BENCH_B r0 all
    # ('GBDT5', 'EXP_BENCH', 'd308acba0eaf453b9f5acfc309b05c46'), # GBDT BENCH_B r1 all
    ('GBDT6', 'EXP_BENCH', 'ec4284016f6d44d897969410a5ef3ec0'), # GBDT BENCH_NOW r0 csi300
    # ('GBDT7', 'EXP_BENCH', 'c9dc8e870efc459396ea77850a7bc099'), # GBDT BENCH_NOW r1 csi300
    ('GBDT8', 'EXP_BENCH', '1f1711ba5caa4fd7a298196fd6a57558'), # GBDT BENCH_A r0 csi300
    # ('GBDT9', 'EXP_BENCH', 'a79446d3724942c58ed14c59622fbe2c'), # GBDT BENCH_A r1 csi300
    ('GBDT10', 'EXP_BENCH', '74d88afd5cb3448d831402b4e7571dc6'), # GBDT BENCH_B r0 csi300
    # ('GBDT5', 'EXP_BENCH', '58317fb6231c4ba0b4d8cecc07db822d'), # GBDT BENCH_B r1 csi300
]

# EXP_NAME, rid = REC_LIST[0]
TOPK = 5
NDROP = 1
HT = 0

info = {
    'ALGO': ['GBDT'],
    'market': ['csi300_ext'], 
    # 'market': ['all'], 
    'benchmark': ["SH000300"], 
    'feat': ["Alpha158"], 
    # 'feat': ["Alpha360"], 
    'label': ['r0'],
    'params': [f'topk {TOPK} ndrop {NDROP} HT {HT}']
}

In [3]:
import qlib
import pandas as pd
from pandas import Timestamp
import json
from qlib.constant import REG_CN
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict
from qlib.data.filter import NameDFilter
from qlib.backtest.position import Position
from qlib.data import D

import copy
import pprint
import warnings
warnings.filterwarnings('ignore')

provider_uri = "/data/linq/.qlib/qlib_data/cn_data"  # target_dir
qlib.init(provider_uri=provider_uri, region=REG_CN)

from lilab.qlib.utils.tools import normalize_position_history, load_position_text, load_position_history, save_position_history, fill_price_position_history
from lilab.qlib.backtest.benchmark import BENCH_Step

/home/linq/anaconda3/envs/qlib39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2941890:MainThread](2024-12-11 21:25:01,790) INFO - qlib.Initialization - [config.py:416] - default_conf: client.


[2941890:MainThread](2024-12-11 21:25:01,794) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.


[2941890:MainThread](2024-12-11 21:25:01,795) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/data/linq/.qlib/qlib_data/cn_data')}


Record position history

In [4]:
cur_pos = load_position_text(pos_txt)
norm_cur_pos = normalize_position_history(cur_pos)
norm_cur_pos = fill_price_position_history(norm_cur_pos)
try:
    pos_history = load_position_history()
except:
    pos_history = {}
pos_history.update(norm_cur_pos)
save_position_history(pos_history)
dt, pos = next(iter(cur_pos.items()))
account = pos.position
dt, norm_pos = next(iter(norm_cur_pos.items()))
norm_account = pos.position
hold_lst = pos.get_stock_list()
print(dt)
print(account)
print(norm_pos)
print(hold_lst)

2024-12-11 09:00:00
{'SH600570': {'amount': 800}, 'SZ002459': {'amount': 1300}, 'SH600104': {'amount': 1100}, 'SH600588': {'amount': 1500}, 'SH600233': {'amount': 1000}, 'cash': 316.0}
{'_settle_type': 'None', 'position': {'SZ002459': {'amount': 17480, 'price': 1.1653757095336914}, 'SH600233': {'amount': 7140, 'price': 2.0420126914978027}, 'SH600570': {'amount': 3727, 'price': 6.6138176918029785}, 'SH600104': {'amount': 12461, 'price': 1.564157485961914}, 'SH600588': {'amount': 7188, 'price': 2.616530179977417}, 'cash': 316.0, 'now_account_value': 98215.02192354202}, 'init_cash': 316.0}
['SZ002459', 'SH600233', 'SH600570', 'SH600104', 'SH600588']


Prepare dataset

In [5]:
nameDFilter = NameDFilter(name_rule_re='(SH60[0-9]{4})|(SZ00[0-9]{4})')
filter_pipe=[nameDFilter]
benchmark = BENCH_Step(test_split,
                        market=info['market'][0], \
                        benchmark=info['benchmark'][0], \
                        feat=info['feat'][0], \
                        label=info['label'][0], \
                        account=norm_account, \
                        filter_pipe=filter_pipe)

bt_benchmark = BENCH_Step(bt_split,
                        market=info['market'][0], \
                        benchmark=info['benchmark'][0], \
                        feat=info['feat'][0], \
                        label=info['label'][0], \
                        account=norm_account, \
                        filter_pipe=filter_pipe)

In [6]:
dataset = init_instance_by_config(benchmark.dataset_config)
bt_dataset = init_instance_by_config(bt_benchmark.dataset_config)
# dataset.prepare('test')

[2941890:MainThread](2024-12-11 21:25:48,217) INFO - qlib.timer - [log.py:127] - Time cost: 45.978s | Loading data Done


[2941890:MainThread](2024-12-11 21:25:48,221) INFO - qlib.timer - [log.py:127] - Time cost: 0.002s | DropnaLabel Done


[2941890:MainThread](2024-12-11 21:25:48,227) INFO - qlib.timer - [log.py:127] - Time cost: 0.004s | CSZScoreNorm Done


[2941890:MainThread](2024-12-11 21:25:48,228) INFO - qlib.timer - [log.py:127] - Time cost: 0.009s | fit & process data Done


[2941890:MainThread](2024-12-11 21:25:48,229) INFO - qlib.timer - [log.py:127] - Time cost: 45.991s | Init data Done


[2941890:MainThread](2024-12-11 21:26:34,024) INFO - qlib.timer - [log.py:127] - Time cost: 45.792s | Loading data Done


[2941890:MainThread](2024-12-11 21:26:34,031) INFO - qlib.timer - [log.py:127] - Time cost: 0.002s | DropnaLabel Done


[2941890:MainThread](2024-12-11 21:26:34,046) INFO - qlib.timer - [log.py:127] - Time cost: 0.014s | CSZScoreNorm Done


[2941890:MainThread](2024-12-11 21:26:34,047) INFO - qlib.timer - [log.py:127] - Time cost: 0.021s | fit & process data Done


[2941890:MainThread](2024-12-11 21:26:34,048) INFO - qlib.timer - [log.py:127] - Time cost: 45.817s | Init data Done


In [7]:
label_df = bt_dataset.prepare("test")
label_df = pd.DataFrame(label_df.loc[:, 'LABEL0'])
label_df.columns = ['label']
# label_df.index = label_df.index.droplevel(0)
label_df

label
datetime   instrument          
2024-12-02 SH600000    0.016860
           SH600009   -0.000560
           SH600010   -0.005051
           SH600011    0.005666
           SH600015    0.030956
...                         ...
2024-12-10 SZ002841   -0.016533
           SZ002916    0.001314
           SZ002920   -0.017960
           SZ002938    0.006693
           SZ003816    0.000000

[1799 rows x 1 columns]

Load and run model

In [8]:
def simp_eval(res_df, k, t=None, day=None):
    if day:
        date = str(cal.iloc[-day,0])
        res_df = res_df[res_df.index.get_level_values('datetime') >= date]
    # Step 1: Group by multiple indices
    grouped = res_df.groupby(['datetime'])

    # Step 2: Sort by 'value' within each group in descending order
    sorted_df = grouped.apply(lambda x: x.sort_values(by='score', ascending=False))
    sorted_df['win'] = sorted_df['label'] * sorted_df['score'] > 0
    # Step 3: Select the top 5 rows for each group
    top_k_per_group = sorted_df.groupby(['datetime']).head(k)
    if t:
        top_k_per_group = top_k_per_group.tail(t)
    # print(top_k_per_group)
    top_k_ic = top_k_per_group['label'].corr(top_k_per_group['score'])
    top_k_win = 1.0 * top_k_per_group['win'].sum() / len(top_k_per_group)
    return top_k_ic, top_k_win

def run_model(model):
    BT_EXP_NAME = 'backtest'

    # backtest and analysis
    with R.start(experiment_name=BT_EXP_NAME, uri='./mlrun'):    
        recorder = R.get_recorder()
        ba_rid = R.get_recorder().id
        sr = SignalRecord(model, bt_dataset, recorder)
        sr.generate()
    with R.start(experiment_name=BT_EXP_NAME, uri='./mlrun'):
        recorder = R.get_recorder(recorder_id=ba_rid)
        bt_pred_df = recorder.load_object("pred.pkl")
    res_df = pd.concat([label_df, bt_pred_df], axis=1, sort=True).reindex( label_df.index)

    def eval_day(day=None):
        top5ic, top5win = simp_eval(res_df, 5, day=day)
        top10ic, top10win = simp_eval(res_df, 10, 5, day=day)
        top15ic, top15win = simp_eval(res_df, 15, 5, day=day)
        top20ic, top20win = simp_eval(res_df, 20, 5, day=day)
        topic, topwin = simp_eval(res_df, 20, day=day)
        return [topic, topwin, top5ic, top5win, top10ic, top10win, top15ic, top15win, top20ic, top20win]
    day5metrics = eval_day(5)
    day10metrics = eval_day(10)
    day20metrics = eval_day(20)
    EXP_NAME = 'realworld_position_maker'

    # backtest and analysis
    with R.start(experiment_name=EXP_NAME, uri='./mlrun'):    
        recorder = R.get_recorder()
        ba_rid = R.get_recorder().id
        sr = SignalRecord(model, dataset, recorder)
        sr.generate()
    with R.start(experiment_name=EXP_NAME, uri='./mlrun'):
        recorder = R.get_recorder(recorder_id=ba_rid)
        pred_df = recorder.load_object("pred.pkl")

    return day5metrics, day10metrics, day20metrics, pred_df

# Run

In [9]:
info_dict5 = {}
info_dict10 = {}
info_dict20 = {}
pred_lst = []

for NAME, EXP_NAME, rid in REC_LIST:
    with R.start(experiment_name=EXP_NAME, uri='./mlrun'):
        recorder = R.get_recorder(recorder_id=rid)
        model = recorder.load_object("trained_model")
    day5metrics, day10metrics, day20metrics, pred_df = run_model(model)
    info_dict5[NAME] = day5metrics
    info_dict10[NAME] = day10metrics
    info_dict20[NAME] = day20metrics
    pred_lst.append(pred_df)

[2941890:MainThread](2024-12-11 21:26:34,102) INFO - qlib.workflow - [exp.py:258] - Experiment 2 starts running ...


[2941890:MainThread](2024-12-11 21:26:34,182) INFO - qlib.workflow - [recorder.py:341] - Recorder 57e5cb9d958e4288aaf8c6c80c564f89 starts running under Experiment 2 ...


[2941890:MainThread](2024-12-11 21:26:36,069) INFO - qlib.timer - [log.py:127] - Time cost: 0.001s | waiting `async_log` Done


[2941890:MainThread](2024-12-11 21:26:36,080) INFO - qlib.workflow - [exp.py:258] - Experiment 4 starts running ...


[2941890:MainThread](2024-12-11 21:26:36,091) INFO - qlib.workflow - [recorder.py:341] - Recorder f333bbd5df914413a5d8129f12fbac49 starts running under Experiment 4 ...


...

[2941890:MainThread](2024-12-11 21:26:40,886) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done


[2941890:MainThread](2024-12-11 21:26:40,891) INFO - qlib.workflow - [exp.py:258] - Experiment 3 starts running ...


[2941890:MainThread](2024-12-11 21:26:40,906) INFO - qlib.workflow - [recorder.py:341] - Recorder 718088992d4940cda263e3241791269a starts running under Experiment 3 ...


[2941890:MainThread](2024-12-11 21:26:40,998) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done


'The following are prediction results of the LGBModel model.'
                          score
datetime   instrument          
2024-12-11 SH600000   -0.090175
           SH600009    0.015240
           SH600010   -0.217294
           SH600011   -0.026459
           SH600015   -0.023623


In [10]:
info_df5 = pd.DataFrame(info_dict5, index=['TOP20IC', 'TOP20WIN', '0-5IC', '0-5WIN', '5-10IC', '5-10WIN', '10-15IC', '10-15WIN', '15-20C', '15-20WIN'])
report_info5 = info_df5

info_df10 = pd.DataFrame(info_dict10, index=['TOP20IC', 'TOP20WIN', '0-5IC', '0-5WIN', '5-10IC', '5-10WIN', '10-15IC', '10-15WIN', '15-20C', '15-20WIN'])
report_info10 = info_df10

info_df20 = pd.DataFrame(info_dict20, index=['TOP20IC', 'TOP20WIN', '0-5IC', '0-5WIN', '5-10IC', '5-10WIN', '10-15IC', '10-15WIN', '15-20C', '15-20WIN'])
report_info20 = info_df20

In [11]:
pred_df = pd.concat(pred_lst, axis=1)
pred_df.columns = name_lst = [r[0] for r in REC_LIST]
_pred_df = pred_df

pred_df['mean'] = _pred_df.mean(axis=1)
pred_df['std'] = _pred_df.std(axis=1)
pred_df['score'] = pd.concat([pred_df[n]*info_dict20[n][0] for n in name_lst], axis=1).sum(axis=1)
# pred_df

In [12]:
rank_df = pred_df.droplevel(0).sort_values(by='score', ascending=False)
fields = ['$close/$factor']
_df = D.features(rank_df.index.values, fields, start_time=TODAY, end_time=TODAY, freq='day')
_df.columns = ['close']
_df = _df.droplevel(1)
rank_df = pd.concat([rank_df,_df],axis=1,join='inner')
report_top = rank_df.head(20)

In [13]:
hold_lst

['SZ002459', 'SH600233', 'SH600570', 'SH600104', 'SH600588']

In [14]:
pos_rank = rank_df.sort_values(by='score', ascending=True)
pos_rank = pos_rank[pos_rank.index.isin(hold_lst)]
report_pos = pos_rank

In [15]:
sty_report_info5 = report_info5.style.background_gradient(axis=0)
sty_report_info5

,GBDT0,GBDT1,GBDT2,GBDT4,GBDT6,GBDT8,GBDT10
TOP20IC,-0.116405,-0.097999,-0.176569,-0.129401,-0.263866,-0.165495,-0.197615
TOP20WIN,0.512500,0.562500,0.512500,0.500000,0.562500,0.550000,0.600000
0-5IC,0.139732,0.074524,-0.051629,-0.018790,-0.231766,-0.134125,-0.075129
0-5WIN,0.400000,0.400000,0.450000,0.450000,0.450000,0.400000,0.450000
5-10IC,-0.293747,-0.198808,-0.639140,0.952562,-0.906629,0.365052,0.428400
5-10WIN,0.400000,0.000000,0.400000,0.000000,0.200000,0.200000,0.400000
10-15IC,-0.655576,-0.448134,-0.408095,0.515664,-0.905053,0.610657,-0.872448
10-15WIN,0.200000,0.600000,0.400000,0.400000,0.400000,0.400000,0.400000
15-20C,0.218905,-0.750942,-0.513839,0.333323,0.209216,-0.894879,-0.033144
15-20WIN,0.800000,0.800000,0.600000,0.200000,0.200000,0.400000,1.000000


In [16]:
sty_report_info10 = report_info10.style.background_gradient(axis=0)
sty_report_info10

,GBDT0,GBDT1,GBDT2,GBDT4,GBDT6,GBDT8,GBDT10
TOP20IC,0.283245,0.138847,0.297928,0.325754,0.198274,0.179784,0.128890
TOP20WIN,0.492857,0.521429,0.485714,0.471429,0.500000,0.542857,0.500000
0-5IC,0.576081,0.515946,0.529748,0.485526,0.494829,0.480605,0.342633
0-5WIN,0.371429,0.371429,0.485714,0.457143,0.342857,0.400000,0.400000
5-10IC,-0.293747,-0.198808,-0.639140,0.952562,-0.906629,0.365052,0.428400
5-10WIN,0.400000,0.000000,0.400000,0.000000,0.200000,0.200000,0.400000
10-15IC,-0.655576,-0.448134,-0.408095,0.515664,-0.905053,0.610657,-0.872448
10-15WIN,0.200000,0.600000,0.400000,0.400000,0.400000,0.400000,0.400000
15-20C,0.218905,-0.750942,-0.513839,0.333323,0.209216,-0.894879,-0.033144
15-20WIN,0.800000,0.800000,0.600000,0.200000,0.200000,0.400000,1.000000


In [17]:
sty_report_info20 = report_info20.style.background_gradient(axis=0)
sty_report_info20

,GBDT0,GBDT1,GBDT2,GBDT4,GBDT6,GBDT8,GBDT10
TOP20IC,0.283245,0.138847,0.297928,0.325754,0.198274,0.179784,0.128890
TOP20WIN,0.492857,0.521429,0.485714,0.471429,0.500000,0.542857,0.500000
0-5IC,0.576081,0.515946,0.529748,0.485526,0.494829,0.480605,0.342633
0-5WIN,0.371429,0.371429,0.485714,0.457143,0.342857,0.400000,0.400000
5-10IC,-0.293747,-0.198808,-0.639140,0.952562,-0.906629,0.365052,0.428400
5-10WIN,0.400000,0.000000,0.400000,0.000000,0.200000,0.200000,0.400000
10-15IC,-0.655576,-0.448134,-0.408095,0.515664,-0.905053,0.610657,-0.872448
10-15WIN,0.200000,0.600000,0.400000,0.400000,0.400000,0.400000,0.400000
15-20C,0.218905,-0.750942,-0.513839,0.333323,0.209216,-0.894879,-0.033144
15-20WIN,0.800000,0.800000,0.600000,0.200000,0.200000,0.400000,1.000000


In [18]:
color_lst = [n for n in report_pos.columns if n not in ['close']]
# color_lst

In [19]:
sty_report_pos = report_pos.style.background_gradient(cmap='Blues_r', subset=color_lst)
sty_report_pos

,GBDT0,GBDT1,GBDT2,GBDT4,GBDT6,GBDT8,GBDT10,mean,std,score,close
instrument,,,,,,,,,,,
SH600104,-0.098783,0.063942,-0.209102,-0.251170,-0.066828,-0.077364,-0.184406,-0.117673,0.098695,-0.214146,17.719999
SH600570,-0.036488,0.034446,-0.193811,-0.106501,-0.030736,-0.110654,-0.220868,-0.094945,0.084763,-0.152443,30.820000
SH600233,-0.042048,-0.016691,-0.062771,-0.067090,-0.059841,-0.055254,-0.023180,-0.046696,0.018498,-0.079570,14.579999
SH600588,-0.024862,0.040568,-0.083755,-0.069144,-0.029930,-0.070510,-0.081709,-0.045620,0.041464,-0.078029,12.540000
SZ002459,0.042205,0.014162,0.159591,0.177756,0.073762,0.117429,0.281943,0.123836,0.084873,0.191449,15.669999


In [20]:
sty_report_top0 = report_top.head(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top1 = report_top.tail(10).style.background_gradient(axis=0, subset=color_lst)

In [21]:
sty_report_top0

,GBDT0,GBDT1,GBDT2,GBDT4,GBDT6,GBDT8,GBDT10,mean,std,score,close
instrument,,,,,,,,,,,
SH600732,0.061153,0.032611,0.058294,0.169086,0.143052,0.193041,0.341945,0.142740,0.099156,0.201440,13.550000
SH600438,0.059130,0.017226,0.146164,0.165251,0.107720,0.135309,0.278114,0.129845,0.077189,0.198048,25.260000
SZ002459,0.042205,0.014162,0.159591,0.177756,0.073762,0.117429,0.281943,0.123836,0.084873,0.191449,15.669999
SH601100,0.072760,0.020175,0.138803,0.069400,0.102297,0.167267,0.182782,0.107640,0.054156,0.161284,57.199997
SH603019,0.023012,0.010215,0.086550,0.180503,0.085540,0.109662,0.185919,0.097343,0.063567,0.153161,73.720001
SH600745,0.064470,0.034757,0.134318,0.150214,0.065036,0.076941,0.110817,0.090936,0.038756,0.153047,36.040001
SH600048,0.059288,-0.004718,0.112340,0.144955,0.037395,0.076947,0.234678,0.094412,0.072867,0.148323,10.160000
SH601865,0.043623,0.003502,0.102191,0.148160,0.058807,0.074147,0.199810,0.090034,0.061466,0.142295,24.070000
SH601816,0.082226,0.043740,0.132943,0.065942,0.053698,0.134357,0.105242,0.088307,0.034061,0.138818,6.220000


In [22]:
sty_report_top1

,GBDT0,GBDT1,GBDT2,GBDT4,GBDT6,GBDT8,GBDT10,mean,std,score,close
instrument,,,,,,,,,,,
SH601998,0.104374,0.049967,0.178687,0.022821,0.086610,0.103320,0.000199,0.077997,0.055333,0.132944,6.940000
SZ000002,0.059188,-0.007218,0.075758,0.130701,0.046444,0.070454,0.198107,0.081919,0.060674,0.128318,8.650001
SH603986,0.050608,0.004974,0.061143,0.136690,0.044561,0.072094,0.173330,0.077629,0.053521,0.121906,84.209999
SZ002129,0.048354,0.007844,0.069784,0.132141,0.054902,0.071724,0.150629,0.076483,0.045719,0.121816,9.990001
SH601888,0.038952,-0.022032,0.074912,0.081347,0.056243,0.105002,0.256989,0.084487,0.079729,0.119944,71.849998
SH600372,0.043602,0.604621,0.080224,-0.006917,0.003264,0.005142,-0.000499,0.104205,0.206321,0.119455,nan
SH601808,0.062836,-0.009059,0.106529,0.113922,0.037889,0.065460,0.086947,0.066360,0.039300,0.115876,15.130000
SZ002415,0.062584,0.004261,0.036414,0.075343,0.069627,0.079576,0.254328,0.083162,0.074055,0.114602,31.509998
SZ000999,0.055909,-0.012868,0.083641,0.081182,0.082928,0.083644,0.102230,0.068095,0.035342,0.110070,45.250000


In [23]:
import dataframe_image as dfi

infodf = pd.DataFrame({'label': ['Model update date', 'Prediction generation date'],
                       'date': ['2024-12-11', TODAY]})

dfi.export(infodf, './tmp/ensemble_info.png',table_conversion='matplotlib')

dfi.export(sty_report_info5, './tmp/ensemble_performance5.png',table_conversion='matplotlib')
dfi.export(sty_report_info10, './tmp/ensemble_performance10.png',table_conversion='matplotlib')
dfi.export(sty_report_info20, './tmp/ensemble_performance20.png',table_conversion='matplotlib')
dfi.export(sty_report_top0, './tmp/ensemble_top0.png',table_conversion='matplotlib')
dfi.export(sty_report_top1, './tmp/ensemble_top1.png',table_conversion='matplotlib')
dfi.export(sty_report_pos, './tmp/ensemble_watch.png',table_conversion='matplotlib')